# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
output_data_file = "output_data/cities.csv"

hotel_data = pd.read_csv("../starter_code/city_data.csv")

hotel_df = pd.DataFrame(hotel_data)
hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Yessentukskaya,44.0328,42.8813,35.6,93,90,17.90
1,1,Vaini,-21.2000,-175.2000,80.6,78,75,8.05
2,2,Barrow,71.2906,-156.7887,-9.4,64,20,6.91
3,3,Punta Arenas,-53.1500,-70.9167,51.8,71,75,6.91
4,4,Ardakān,32.3100,54.0175,55.4,16,0,2.30


In [30]:
gmaps.configure(api_key = g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
locations = hotel_df[['Latitude', 'Longitude']]

humidity = hotel_df['Humidity'].astype(float)

In [45]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False,
                                    max_intensity=100, point_radius=3.0)

fig.add_layer(heatmap_layer)

plt.savefig("vacationheatmap.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
ideal_hotel_df = hotel_df.loc[(hotel_df["Wind Speed"] <= 10) & (hotel_df["Cloudiness"] == 0) & \
                                  (hotel_df["Temperature"] >= 70) & (hotel_df["Temperature"] <= 80)].dropna()

ideal_hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
81,81,Vicuña,-30.0319,-70.7081,70.00,34,0,1.34
172,172,Amalāpuram,16.5833,82.0167,71.65,80,0,1.88
176,176,Salalah,17.0151,54.0924,75.20,57,0,3.44
219,219,Turbat,26.0023,63.0440,72.18,56,0,2.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
ideal_hotel_df['Hotel Name'] = ""
ideal_hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
81,81,Vicuña,-30.0319,-70.7081,70.00,34,0,1.34,
172,172,Amalāpuram,16.5833,82.0167,71.65,80,0,1.88,
176,176,Salalah,17.0151,54.0924,75.20,57,0,3.44,
219,219,Turbat,26.0023,63.0440,72.18,56,0,2.73,


In [35]:
for index, row in ideal_hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        ideal_hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        ideal_hotel_df.loc[index, "Hotel Name"] = "NaN"
        
ideal_hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
81,81,Vicuña,-30.0319,-70.7081,70.00,34,0,1.34,Hotel Tukun Mapu
172,172,Amalāpuram,16.5833,82.0167,71.65,80,0,1.88,swayamprabha service apartments
176,176,Salalah,17.0151,54.0924,75.20,57,0,3.44,HYATT SALALAH HOTEL
219,219,Turbat,26.0023,63.0440,72.18,56,0,2.73,Khand a Hotel


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_hotel_df.iterrows()]
locations = ideal_hotel_df[["Latitude", "Longitude"]]

In [43]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))